<a href="https://colab.research.google.com/github/JoshJoseph1234/Data-Mining-Assignment/blob/main/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!cp "/content/drive/My Drive/dataset.zip" "/content/dataset.zip"


In [3]:
import zipfile

zip_path = "/content/dataset.zip"
extract_path = "/content/dataset"

try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Extraction successful!")
except zipfile.BadZipFile:
    print("Error: The file is not a valid ZIP archive.")


Extraction successful!


In [4]:
import os

dataset_path = "/content/dataset"
print("Folders in dataset:", os.listdir(dataset_path))


Folders in dataset: ['dataset']


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = "/content/dataset/dataset"

# Define parameters
img_size = (28, 28)  # Resize images to 28x28 to match MNIST format
batch_size = 32

# Load training images using ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalize pixel values

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    color_mode="grayscale",  # Convert to grayscale
    batch_size=batch_size,
    class_mode="categorical",  # Multi-class classification (digits 0-9)
    subset="training"
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)


Found 4247 images belonging to 10 classes.
Found 1057 images belonging to 10 classes.


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

model = Sequential([
    Flatten(input_shape=(28, 28, 1)),  # Convert 28x28 image into a 1D vector
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # Output layer (10 classes)
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, validation_data=val_data, epochs=5)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
133/133 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - accuracy: 0.1167 - loss: 2.4683 - val_accuracy: 0.1202 - val_loss: 2.2815
Epoch 2/5
133/133 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.1684 - loss: 2.2462 - val_accuracy: 0.2233 - val_loss: 2.1643
Epoch 3/5
133/133 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - accuracy: 0.2236 - loss: 2.1491 - val_accuracy: 0.2460 - val_loss: 2.1192
Epoch 4/5
133/133 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.2737 - loss: 2.0740 - val_accuracy: 0.3330 - val_loss: 2.0973
Epoch 5/5
133/133 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.3163 - loss: 2.0069 - val_accuracy: 0.3359 - val_loss: 1.9619


In [8]:
loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.3436 - loss: 1.9519
Validation Loss: 1.9619
Validation Accuracy: 0.3359


In [10]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img_path = "/content/images.png"  # Update with your image path
img = image.load_img(img_path, target_size=(28, 28), color_mode='grayscale')  # Load image and resize to 28x28

# Convert image to array and normalize it
img_array = image.img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]

# Add batch dimension (the model expects a batch of images)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension (shape becomes (1, 28, 28, 1))

# Make prediction
predictions = model.predict(img_array)

# Get predicted class (digit)
predicted_class = np.argmax(predictions, axis=1)
print(f"Predicted Class: {predicted_class[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Predicted Class: 3


In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization

# Define dataset path
dataset_path = "/content/dataset/dataset"

# Define image size and batch size
img_size = (28, 28)
batch_size = 32

# Improved Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,  # Rotate images slightly
    width_shift_range=0.2,  # Shift images left/right
    height_shift_range=0.2,  # Shift images up/down
    zoom_range=0.2,  # Zoom in on images
    shear_range=0.2,  # Apply shearing
    horizontal_flip=False,  # Disable horizontal flipping for digit classification
    fill_mode='nearest'
)

# Load training and validation data
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Define the model (without changing architecture)
model = Sequential([
    Flatten(input_shape=(28, 28, 1)),  # Convert 28x28 image into a 1D vector
    BatchNormalization(),  # Added batch normalization
    Dense(128, activation='relu'),
    BatchNormalization(),  # Added batch normalization
    Dense(10, activation='softmax')  # Output layer (10 classes)
])

# Compile model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks to improve training
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
model.fit(train_data, validation_data=val_data, epochs=20, callbacks=[early_stopping, reduce_lr])

# Evaluate the model
loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


Found 4247 images belonging to 10 classes.
Found 1057 images belonging to 10 classes.
Epoch 1/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 10s 65ms/step - accuracy: 0.1610 - loss: 2.7171 - val_accuracy: 0.1079 - val_loss: 4.3088 - learning_rate: 5.0000e-04
Epoch 2/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 9s 65ms/step - accuracy: 0.2211 - loss: 2.2679 - val_accuracy: 0.1788 - val_loss: 2.6046 - learning_rate: 5.0000e-04
Epoch 3/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - accuracy: 0.2796 - loss: 2.1165 - val_accuracy: 0.2725 - val_loss: 2.1438 - learning_rate: 5.0000e-04
Epoch 4/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - accuracy: 0.3041 - loss: 2.0370 - val_accuracy: 0.3396 - val_loss: 1.9721 - learning_rate: 5.0000e-04
Epoch 5/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.3198 - loss: 1.9685 - val_accuracy: 0.3415 - val_loss: 1.9313 - learning_rate: 5.0000e-04
Epoch 6/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - accuracy: 0.3327 - loss: 1.9285 - val_accuracy: 0.3756 - val_loss: 1.8577

In [14]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_digit(model, img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(28, 28), color_mode='grayscale')

    # Convert image to array and normalize
    img_array = image.img_to_array(img) / 255.0  # Normalize pixel values to [0,1]

    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)  # Shape: (1, 28, 28, 1)

    # Make prediction
    predictions = model.predict(img_array)

    # Get predicted class (digit)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Print prediction confidence
    confidence = np.max(predictions) * 100
    print(f"Predicted Class: {predicted_class} (Confidence: {confidence:.2f}%)")

    return predicted_class

# Example Usage:
img_path = "/content/images.png"  # Update with your image path
predict_digit(model, img_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted Class: 1 (Confidence: 17.95%)


np.int64(1)